# 0. Project Setup and Authentication

**Objective:** Configure the environment and authenticate with Google Cloud to access the MIMIC-III dataset on BigQuery.

This notebook handles the necessary setup for running the project in both Google Colab and a local environment.

### 1. Import Libraries

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

# A robust way to check if we are in Google Colab
try:
    from google.colab import auth
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

### 2. Configure Google Cloud Project

In [ ]:
project_id = 'deep-atlas-413309' # Yair Bigquery Project ID
# project_id = 'light-legend-457315-k3' # Lital Bigquery Project ID

### 3. Authenticate and Initialize Client

The following cell handles authentication. 
- **In Google Colab:** It will prompt you to authenticate with your Google account.
- **Locally:** It will use the credentials from the `gcloud` CLI. If you haven't authenticated locally before, please run the following command in your terminal and follow the instructions:

  ```bash
  gcloud auth application-default login
  ```

In [ ]:
if IN_COLAB:
    print("Authenticating in Google Colab...")
    auth.authenticate_user()
else:
    print("Using local Google Cloud SDK credentials.")

client = bigquery.Client(project=project_id)
print(f"BigQuery client initialized for project: {client.project}")
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## Admissions

In [ ]:
# @title Query DB
hospquery = \
"""
SELECT admissions.subject_id, admissions.hadm_id
, admissions.admittime, admissions.dischtime
, admissions.ethnicity, admissions.deathtime
, admissions.insurance
, patients.gender, patients.dob
FROM `physionet-data.mimiciii_clinical.admissions` admissions
INNER JOIN `physionet-data.mimiciii_clinical.patients` patients
    ON admissions.subject_id = patients.subject_id
WHERE admissions.has_chartevents_data = 1
ORDER BY admissions.subject_id, admissions.hadm_id, admissions.admittime;
"""
hosps = client.query(hospquery).result().to_dataframe().rename(str.lower, axis='columns')
hosps.head()